In [362]:
from pippi.soundbuffer import SoundBuffer
from pippi import dsp,fx
import numpy as np
import pandas as pd
import sounddevice as sd
import helpers as hp
import param_generation as pg
import _pickle as pickle
from IPython.display import Audio
import librosa
import librosa.feature
from feature_extraction import pytorch_models as tm
import optuna
import torch
SR = 44100


Load models

In [363]:
imp.reload(tm)
cnet_dvn = tm.Transformer_DVN(attention_dropout=0.3,d_model=120,heads=30,encoding_layers=12,pool_dim=2,)
cnet_dvn = cnet_dvn.load_from_checkpoint(
    "models/transformer/DVNepoch=41-val_accuracy=0.991-val_loss=0.048.ckpt")

cnet_dvd = tm.Transformer_DVD(attention_dropout=0.3,d_model=100,heads=20,encoding_layers=12,pool_dim=1,pct_start=0.05,max_lr=1e-4,max_momentum=0.95,epochs=50)
cnet_dvd = cnet_dvd.load_from_checkpoint(
    "models/transformer/DVDepoch=59-val_accuracy=0.917-val_auc=0.9779-val_loss=0.327.ckpt")

In [364]:
out = dsp.buffer(length=1,channels=1)
p = pg.StackParams()


In [365]:
s = pg.Synth(p)
out.dub(s.buff,p.getStart())
Audio(hp.memToAud(out),rate=SR, autoplay=True)

In [196]:
%%time
#generate 1 sound
import imp
imp.reload(pg)
imp.reload(hp)

out = dsp.buffer(length=1,channels=1)
params = []
stack_num = np.random.choice([2])
for i in range(stack_num): 
    p = pg.StackParams()
    s = pg.Synth(p)
    out.dub(s.buff,p.getStart())
    params.append(p)
out = fx.norm(out,1)

# z=librosa.feature.rms(hp.memToAud(out))

# print(np.sum(z),hp.memToAud(out).shape)
# hp.specShow(out)
Audio(hp.memToAud(out),rate=SR, autoplay=True)


CPU times: user 22.7 ms, sys: 0 ns, total: 22.7 ms
Wall time: 22.1 ms


In [225]:
class StackParams():
    def __init__(self,name="pset"):
        self.oscType=rd.choice([0,1,2],p=[0.8,0.1,0.1])
        self.isNoise=rd.choice([0,1],p=[0.5,0.5])     
        self.A=rd.randint(len(a_d_s_r))
        self.D=rd.randint(len(a_d_s_r))
        self.S=rd.randint(len(a_d_s_r))
        self.R=rd.randint(len(a_d_s_r)) 
        self.pitch_0=rd.choice(p0_pitches)
        self.pitch_1=rd.choice(p1_pitches)
        self.pitch_2=rd.choice(p2_pitches)
        self.pitch_3=rd.choice(p3_pitches) 
        self.bpCutLow=rd.choice(bp_pitches)
        self.bpCutHigh=rd.randint(self.bpCutLow,num_notes)
        self.bpOrder=rd.randint(len(filter_orders))
        self.amplitude=rd.randint(len(amplitudes))
        self.start=rd.randint(start_spacing)
        self.length=rd.randint(start_spacing-self.start)
    def getOscType(self):
        return osc_types[self.oscType]
    def getPitches(self):
        return all_pitches[self.pitch_0],all_pitches[self.pitch_1],all_pitches[self.pitch_2],all_pitches[self.pitch_3]
    def getBandPass(self):
        return all_pitches[self.bpCutLow],all_pitches[self.bpCutHigh],filter_orders[self.bpOrder]
    def getAmp(self):
        return amplitudes[self.amplitude]
    def getLength(self):
        return starts[self.length]+min_length
    def getStart(self):
        return lengths[self.start]

In [436]:
global sound 
params = dict()
def objective(trial,g_params=z):
    program = pg.StackParams()
    program.oscType = trial.suggest_categorical("OSC Type", [0,1,2])
    program.isNoise = trial.suggest_categorical("Noise Cloud", [0,1])
    
    program.A = trial.suggest_categorical("A", [0,1,2,3])
    program.D = trial.suggest_categorical("D", [0,1,2,3])
    program.S = trial.suggest_categorical("S", [0,1,2,3])
    program.R = trial.suggest_categorical("R", [0,1,2,3])
    
    program.pitch_0 = int(trial.suggest_float("pitch_0", 0.0, 120, step=1))
    program.pitch_1 = int(trial.suggest_float("pitch_1", 0.0, 120, step=1))
    program.pitch_2 = int(trial.suggest_float("pitch_2", 0.0, 120, step=1))
    program.pitch_3 = int(trial.suggest_float("pitch_3", 0.0, 120, step=1))
    
    program.bpCutLow = trial.suggest_categorical('Low Cut', range(0,120))
    program.bpCutHigh = trial.suggest_categorical('High Cut', range(0,120))
    program.bpOrder = trial.suggest_categorical("BP Order", [0,1,2])
    
    program.amplitude = trial.suggest_categorical("Amplitude", [0.3,0.5,0.8,1])
    
    program.start = trial.suggest_categorical("Start",  range(0,10))
    program.length = trial.suggest_categorical("Length", range(0,10))
    
    
    try:
        
        # make the sound 
        global sound 
        out = dsp.buffer(length=1,channels=1)
        s = pg.Synth(program)
        out.dub(s.buff,p.getStart())
        out = fx.norm(out,1)
        sound = hp.memToAud(out)
        # analyze it
        signal = torch.tensor(librosa.resample(sound.reshape(-1),SR,SR//2))[0:SR//4].reshape(1,-1)
        score = cnet_dvn(signal).item()
        print(score)
    except Exception as e:
        sound = [0]*100
        return -10 # bad params 
    return score
study = optuna.create_study()
study.optimize(objective, n_trials=100)

Audio(sound,rate=SR, autoplay=True)

[I 2021-07-30 21:10:45,950] Trial 77 finished with value: -10.0 and parameters: {'OSC Type': 1, 'Noise Cloud': 1, 'A': 2, 'D': 0, 'S': 1, 'R': 1, 'pitch_0': 6.0, 'pitch_1': 9.0, 'pitch_2': 58.0, 'pitch_3': 55.0, 'Low Cut': 84, 'High Cut': 6, 'BP Order': 1, 'Amplitude': 0.5, 'Start': 8, 'Length': 5}. Best is trial 3 with value: -10.0.
[I 2021-07-30 21:10:45,967] Trial 78 finished with value: -10.0 and parameters: {'OSC Type': 1, 'Noise Cloud': 0, 'A': 2, 'D': 0, 'S': 1, 'R': 1, 'pitch_0': 21.0, 'pitch_1': 71.0, 'pitch_2': 41.0, 'pitch_3': 42.0, 'Low Cut': 114, 'High Cut': 31, 'BP Order': 1, 'Amplitude': 0.5, 'Start': 8, 'Length': 3}. Best is trial 3 with value: -10.0.
[I 2021-07-30 21:10:45,984] Trial 79 finished with value: -10.0 and parameters: {'OSC Type': 1, 'Noise Cloud': 0, 'A': 0, 'D': 0, 'S': 1, 'R': 0, 'pitch_0': 72.0, 'pitch_1': 16.0, 'pitch_2': 48.0, 'pitch_3': 36.0, 'Low Cut': 82, 'High Cut': 32, 'BP Order': 1, 'Amplitude': 0.3, 'Start': 8, 'Length': 2}. Best is trial 3 with

-6.668633460998535
-6.752699851989746


[I 2021-07-30 21:10:46,359] Trial 91 finished with value: -10.0 and parameters: {'OSC Type': 0, 'Noise Cloud': 1, 'A': 2, 'D': 1, 'S': 3, 'R': 1, 'pitch_0': 32.0, 'pitch_1': 86.0, 'pitch_2': 21.0, 'pitch_3': 105.0, 'Low Cut': 111, 'High Cut': 22, 'BP Order': 2, 'Amplitude': 0.5, 'Start': 2, 'Length': 6}. Best is trial 3 with value: -10.0.
[I 2021-07-30 21:10:46,457] Trial 92 finished with value: -6.693578243255615 and parameters: {'OSC Type': 1, 'Noise Cloud': 0, 'A': 2, 'D': 3, 'S': 2, 'R': 0, 'pitch_0': 3.0, 'pitch_1': 116.0, 'pitch_2': 44.0, 'pitch_3': 102.0, 'Low Cut': 1, 'High Cut': 22, 'BP Order': 1, 'Amplitude': 0.5, 'Start': 2, 'Length': 3}. Best is trial 3 with value: -10.0.
[I 2021-07-30 21:10:46,572] Trial 93 finished with value: -6.608630657196045 and parameters: {'OSC Type': 0, 'Noise Cloud': 0, 'A': 2, 'D': 2, 'S': 2, 'R': 1, 'pitch_0': 9.0, 'pitch_1': 12.0, 'pitch_2': 13.0, 'pitch_3': 54.0, 'Low Cut': 82, 'High Cut': 104, 'BP Order': 2, 'Amplitude': 0.5, 'Start': 2, 'Len

-6.693578243255615
-6.608630657196045


[I 2021-07-30 21:10:46,669] Trial 94 finished with value: -5.955055236816406 and parameters: {'OSC Type': 1, 'Noise Cloud': 0, 'A': 0, 'D': 2, 'S': 2, 'R': 0, 'pitch_0': 82.0, 'pitch_1': 110.0, 'pitch_2': 50.0, 'pitch_3': 79.0, 'Low Cut': 96, 'High Cut': 118, 'BP Order': 1, 'Amplitude': 1, 'Start': 4, 'Length': 5}. Best is trial 3 with value: -10.0.
[I 2021-07-30 21:10:46,688] Trial 95 finished with value: -10.0 and parameters: {'OSC Type': 1, 'Noise Cloud': 0, 'A': 0, 'D': 2, 'S': 2, 'R': 1, 'pitch_0': 86.0, 'pitch_1': 116.0, 'pitch_2': 3.0, 'pitch_3': 10.0, 'Low Cut': 82, 'High Cut': 32, 'BP Order': 1, 'Amplitude': 1, 'Start': 4, 'Length': 5}. Best is trial 3 with value: -10.0.
[I 2021-07-30 21:10:46,707] Trial 96 finished with value: -10.0 and parameters: {'OSC Type': 2, 'Noise Cloud': 1, 'A': 2, 'D': 3, 'S': 3, 'R': 2, 'pitch_0': 29.0, 'pitch_1': 66.0, 'pitch_2': 70.0, 'pitch_3': 24.0, 'Low Cut': 102, 'High Cut': 72, 'BP Order': 0, 'Amplitude': 0.8, 'Start': 5, 'Length': 4}. Best i

-5.955055236816406
-6.711873531341553


In [416]:
sound

array([0.00000000e+00, 0.00000000e+00, 6.55224037e-37, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])